In [1]:
from z3 import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from IPython.display import Image

from ogrit.decisiontree.dt_goal_recogniser import OcclusionGrit
from ogrit.evaluation.verification import add_goal_tree_model, extract_counter_example, \
            extract_tree_counter_example, add_single_tree_model, verify_proposition
from ogrit.core.data_processing import get_dataset
from ogrit.core.feature_extraction import FeatureExtractor
from ogrit.core.base import get_data_dir, get_img_dir

In [2]:
import z3

# OGRIT

In [3]:
# sanity check
scenario_name = 'heckstrasse'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'exit-left')]

s = Solver()

features, probs, likelihoods = add_goal_tree_model(reachable_goals, s, model)

verify_expr = And(probs[1] > 0., probs[2] > 0.)
s.add(Not(verify_expr))

print(s.check())


unsat


In [4]:
scenario_name = 'heckstrasse'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'exit-left')]

s = Solver()

features, probs, likelihoods = add_goal_tree_model(reachable_goals, s, model)

verify_expr = Implies(And(features[1]['in_correct_lane'],
                          Not(features[2]['in_correct_lane'])),
                      probs[2] < probs[1])
s.add(Not(verify_expr))

print(s.check())

sat


In [5]:
extract_counter_example(s, features, probs, likelihoods)

,1,2
path_to_goal_length,28.0,47.24675
in_correct_lane,True,False
speed,9.0,9.0
acceleration,3.0,3.0
angle_in_lane,2.0,2.0
vehicle_in_front_dist,0.0,0.0
vehicle_in_front_speed,0.0,0.0
oncoming_vehicle_dist,None,None
oncoming_vehicle_speed,None,None
road_heading,1.0,None


Verify that having missing feature does not lead to entropy decrease, all other features equal

In [6]:
scenario_name = 'heckstrasse'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'exit-left')]

s = Solver()

features1, probs1, likelihoods1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2, likelihoods2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

indicator_feature = 'vehicle_in_front_missing'

for goal_idx, goal_type in reachable_goals:
    s.add(And(features1[goal_idx][indicator_feature], \
          Not(features2[goal_idx][indicator_feature])))
    
    for feature_name in features1[goal_idx]:
        if feature_name != indicator_feature:
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])


verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] >= probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] >= probs1[1]))
# 2 should have lower entropy than 1
# missing in 1, not missing in 2
s.add(Not(verify_expr))


print(s.check())

sat


In [7]:
extract_counter_example(s, features1, probs1, likelihoods1)

,1,2
path_to_goal_length,0.0,30.0
in_correct_lane,False,True
speed,10.0,10.0
acceleration,2.0,2.0
angle_in_lane,1.0,1.0
vehicle_in_front_dist,0.0,0.0
vehicle_in_front_speed,0.0,0.0
oncoming_vehicle_dist,0.0,0.0
oncoming_vehicle_speed,0.0,0.0
road_heading,1.0,0.0


In [8]:
extract_counter_example(s, features2, probs2, likelihoods2)

,1,2
path_to_goal_length,0.0,30.0
in_correct_lane,False,True
speed,10.0,10.0
acceleration,2.0,2.0
angle_in_lane,1.0,1.0
vehicle_in_front_dist,0.0,0.0
vehicle_in_front_speed,0.0,0.0
oncoming_vehicle_dist,0.0,0.0
oncoming_vehicle_speed,0.0,0.0
road_heading,1.0,0.0


Straight on goal increases in likelihood, as path to goal length feature is checked in this branch, but not checked in the branch where potentially missing feature is checked.

Being very to goal increases likelihood of striaght on goal - maybe vehicles turning left rarely come this close to the goal.

In [9]:
scenario_name = 'heckstrasse'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'exit-left')]

s = Solver()

features1, probs1, likelihoods1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2, likelihoods2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

indicator_feature = 'oncoming_vehicle_missing'

for goal_idx, goal_type in reachable_goals:
    s.add(And(Not(features1[goal_idx][indicator_feature]), \
          features2[goal_idx][indicator_feature]))
    
    for feature_name in features1[goal_idx]:
        if feature_name != indicator_feature:
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])


verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] >= probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] >= probs1[1]))

s.add(Not(verify_expr))


print(s.check())

unsat


having missing feature does not lead to entropy decrease, all other features equal, if not within 20m of goal

In [10]:
scenario_name = 'heckstrasse'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'exit-left')]

s = Solver()

features1, probs1, likelihoods1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2, likelihoods2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

indicator_feature = 'oncoming_vehicle_missing'

for goal_idx, goal_type in reachable_goals:
    s.add(And(Not(features1[goal_idx][indicator_feature]), \
          features2[goal_idx][indicator_feature]))
    s.add(features1[goal_idx]['path_to_goal_length'] > 20)
    
    for feature_name in features1[goal_idx]:
        if feature_name != indicator_feature:
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])


verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] >= probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] >= probs1[1]))

s.add(Not(verify_expr))


print(s.check())

unsat


if exit number is 4th exit, then likelihood should be lower than if exit number missing 

In [11]:
scenario_name = 'neuweiler'
model = OcclusionGrit.load(scenario_name)
reachable_goals = [(0, 'exit-roundabout'), (1, 'exit-roundabout'),
                   (2, 'exit-roundabout'), (3, 'exit-roundabout')]

s = Solver()

features1, probs1, likelihoods1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2, likelihoods2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

indicator_feature = 'exit_number_missing'

for goal_idx, goal_type in reachable_goals:
    s.add(And(Not(features1[goal_idx][indicator_feature]), \
          features2[goal_idx][indicator_feature]))
    s.add(features1[goal_idx]['path_to_goal_length'] > 20)
    s.add(features1[goal_idx][indicator_feature] == Int(goal_idx+1))
    for feature_name in features1[goal_idx]:
        if feature_name != indicator_feature and feature_name != 'exit_number':
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])


verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] >= probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] >= probs1[1]))

s.add(Not(verify_expr))


print(s.check())

sat


if exit number is 4th exit, then likelihood should be lower than if exit number is 2, if path to goal length is greater than 40m, all other features equal

In [12]:
scenario_name = 'neuweiler'
model = OcclusionGrit.load(scenario_name)
goal_idx = 3
goal_type = 'exit-roundabout'
s = Solver()

features1, likelihood1, prob1 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_1')
features2, likelihood2, prob2 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_2')

for feature_name in features1:
    if feature_name != 'exit_number':
        s.add(features1[feature_name] == features2[feature_name])

s.add(features1['path_to_goal_length'] == 50)
s.add(features1['exit_number'] == 4)
s.add(features2['exit_number'] == 2)

verify_expr = likelihood2 >= likelihood1

s.add(Not(verify_expr))


print(goal_idx, goal_type, s.check())

3 exit-roundabout unsat


if exit number is 4th exit, then likelihood should be lower than if exit number is missing, if path to goal length is 50m, and the vehicle's angle in lane is zero, all other features equal

In [13]:
scenario_name = 'neuweiler'
model = OcclusionGrit.load(scenario_name)
goal_idx = 3
goal_type = 'exit-roundabout'
s = Solver()

features1, likelihood1, prob1 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_1')
features2, likelihood2, prob2 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_2')

for feature_name in features1:
    if feature_name != 'exit_number' and feature_name != 'exit_number_missing':
        s.add(features1[feature_name] == features2[feature_name])

s.add(features1['path_to_goal_length'] == 50)
s.add(features1['angle_in_lane'] == 0)

s.add(features1['exit_number'] == 4)
s.add(features1['exit_number_missing'] == False)
s.add(features2['exit_number_missing'] == True)

verify_expr = likelihood2 >= likelihood1

s.add(Not(verify_expr))


print(goal_idx, goal_type, s.check())

3 exit-roundabout unsat


If a vehicle is stopped at a junction, and oncoming vehicle is missing, then turning right should have higher likelihood than if there is no oncoming vehicle

In [14]:
scenario_name = 'bendplatz'
model = OcclusionGrit.load(scenario_name)
goal_type = 'enter-right'
s = Solver()

features1, likelihood1, prob1 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_1')
features2, likelihood2, prob2 = add_single_tree_model(
    goal_idx, goal_type, s, model, suffix='_2')

for feature_name in features1:
    if feature_name not in ['oncoming_vehicle_missing']:
        s.add(features1[feature_name] == features2[feature_name])

#s.add(features1['path_to_goal_length'] == 10)
s.add(features1['speed'] < 1)
s.add(features1['angle_in_lane'] == 0)

s.add(features1['oncoming_vehicle_speed'] == 20)
s.add(features1['oncoming_vehicle_dist'] == 100)
s.add(features1['oncoming_vehicle_missing'] == False)
s.add(features2['oncoming_vehicle_missing'] == True)

verify_expr = likelihood2 >= likelihood1

s.add(Not(verify_expr))


print(goal_idx, goal_type, s.check())

3 enter-right unsat


In [15]:
pd.merge(extract_tree_counter_example(s, features1, likelihood1).rename('1'),
         extract_tree_counter_example(s, features2, likelihood2).rename('2'),
         right_index=True, left_index=True)

Z3Exception: model is not available